# Contributeurs GitHub

- Récupérer via crawling la liste des 256 top contributors sur cette page https://gist.github.com/paulmillr/2657075 
- En utilisant l'API github https://developer.github.com/v3/ récupérer pour chacun de ces users le nombre moyens de stars des repositories qui leur appartiennent. Pour finir classer ces 256 contributors par leur note moyenne

In [100]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
import json

In [101]:
def get_soup_from_link(link) :
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = requests.get(link, headers=hdr)
    soup = BeautifulSoup(req.content, 'html.parser')
    return soup

In [105]:
username = "Pakiradhaan"
token = open("key.txt", "r").read()

def get_stars_from_user(user) :
    liste = []
    git_api_url = "https://api.github.com/users/" + user + "/repos"
    response = requests.get(git_api_url, auth=(username,token))
    while True :
        json_response = json.loads(response.content)
        if isinstance(json_response, str) :
            print(json_response)
            break
        liste = liste + list(map(lambda x : x.get('stargazers_count'), json_response))
        if 'next' in response.links.keys() :
            response = requests.get(response.links['next']['url'], auth=(username,token))
        else :
            break
    return 0 if (not liste) else np.mean(liste)

In [103]:
soup = get_soup_from_link("https://gist.github.com/paulmillr/2657075")

all_a = list(map(lambda x : x.find('a'), soup.find('table').findAll('tr')))
all_contrib = list(map(lambda x : x.text, [x for x in all_a if x is not None]))

In [111]:
res = list(map(lambda x : get_stars_from_user(x), all_contrib))

In [114]:
df = pd.DataFrame({'Contributeur':all_contrib, 'Note moyenne':res})
df.sort_values(['Note moyenne'], ascending = False).head(10)

,Contributeur,Note moyenne
225,bailicangdu,3502.923077
18,kamranahmedse,2274.113636
142,vinta,1829.843750
117,mrdoob,1597.085714
194,IanLunn,1401.888889
44,sebastianbergmann,1387.816667
230,junyanz,1372.785714
153,nolimits4web,1338.571429
195,ruanyf,1260.584906
109,JakeWharton,1072.850000
